# **Stochastic programming using GUROBI**

Firstly, we need to install and import some libraries with appropriate uses:
*   **numpy**: creating, manipulating matrices and perform matrcies operations.
*  **gurobi**: solving linear program.




In [ ]:
!pip install gurobipy

import numpy as np
import gurobipy as gugu


#**Initiliazation of unchanged values**
Before building the model, we simulate neccessary data. The table below gives description, data type and value or range of these data.


| Name | Description | Data Type | Value or range|
| :-: | :- | :- | |
| n | Number of products | natural | n = 8|
| m | Number of materials | natural | m = 5|
| **b** | Costs of materials | vector 5x1, natural |  $50 \leq b_j \leq 100, \forall j \in J$  |
| **l** | Additional costs | vector 8x1, natural | $10 \leq l_i \leq 25, \forall i \in I$|
| **q** | Sale prices | vector 8x1, natural | $q_i = Ab + l + ϵ_i, 20 \leq ϵ_i \leq 50, \forall i \in I $|
| **s** | Salvage value | vector 5x1, natural | $s_j = b_j - ϵ_j, 10 \leq ϵ_j \leq 20, \forall j \in J $|
|**A** | $a_{ij}$ is the number of material j that a unit of prodcut i requires | matrix 8x5, natural | $ 1 \leq a_{ij} \leq 10, \forall i \in I, \forall j \in J$


In [ ]:
np.random.seed(123)
n = 8
m = 5
b = np.random.randint(low = 50, high = 100, size = (m,1))
l = np.random.randint(low = 10, high = 25, size = (n,1))
s = b - np.random.randint(low = 10, high = 20, size = (m,1))
A = np.random.randint(low = 1, high = 10, size = (n,m)) #8x5
tcostsOfProduct = np.matmul(A, b) + l; # A.b + l
q = tcostsOfProduct + np.random.randint(low = 20, high = 50, size = (n,1))



print("b is ")
print(b)
print("\nl is ")
print(l)
print("\ns is ")
print(s)
print("\nq is ")
print(q)
print("\nMatrix A is")
print(A)


b is 
[[95]
 [52]
 [78]
 [84]
 [88]]

l is 
[[11]
 [13]
 [20]
 [21]
 [19]
 [16]
 [11]
 [10]]

s is 
[[84]
 [33]
 [68]
 [74]
 [69]]

q is 
[[1280]
 [1644]
 [2201]
 [2189]
 [2172]
 [1697]
 [1931]
 [2185]]

Matrix A is
[[4 5 1 1 5]
 [2 8 4 3 5]
 [8 3 5 9 1]
 [8 4 5 7 2]
 [6 7 3 2 9]
 [4 6 1 3 7]
 [3 5 5 7 4]
 [1 7 5 8 7]]


# **Generate Uncertainty**
In this problem, the uncertainty is the demand of products. Therefore, we have 8 random variables $D_i$.
* **Number of scenarios**: S = 2
* **Random variables**: $d_i$ ~ $Bin(10,\frac{1}{2}), \forall i \in I$

In [ ]:
np.random.seed(12)
D_1 = np.random.binomial(10, 0.5, size = (8,1))
D_2 = np.random.binomial(10, 0.5, size = (8,1))

print("D_1 is: ")
print(D_1)
print("\nD_2 is: ")
print(D_2)


D_1 is: 
[[3]
 [6]
 [4]
 [5]
 [2]
 [7]
 [7]
 [2]]

D_2 is: 
[[8]
 [3]
 [4]
 [5]
 [7]
 [7]
 [1]
 [5]]


# **Build Model**
Next, we create a model and then add variables and objective functions. Our linear problem written in standard form is:<br>
  $\textbf{min } b^Tx + E[(l-q)^Tz - s^Ty]$ <br>
such that
\begin{equation}
\begin{cases}
    y = x - A^Tz \\
    0 \leq z \leq D \\
    x,y \geq 0
\end{cases}
\end{equation}






In [ ]:
model = gugu.Model("Industry Manufacturing Problem")
model.setParam('OutputFlag', 0)
x = model.addMVar((m,1), vtype = gugu.GRB.INTEGER, name = "x")
y1 = model.addMVar((m,1), vtype = gugu.GRB.INTEGER, name = "y1")
y2 = model.addMVar((m,1), vtype = gugu.GRB.INTEGER, name = "y2")
z1 = model.addMVar((n,1), vtype = gugu.GRB.INTEGER, name = "z1")
z2 = model.addMVar((n,1), vtype = gugu.GRB.INTEGER, name = "z2")

#Objective function
model.setObjective(b.T @ x + 0.5*(l-q).T @ (z1 + z2) - 0.5*(s.T)@(y1+y2), sense = gugu.GRB.MINIMIZE)

#Add constraints
model.addConstr(y1 == x - A.T @ z1)
model.addConstr(y2 == x - A.T @ z2)
model.addConstr(z1 <= D_1)
model.addConstr(z2 <= D_2)


model.Params.Method = 0
model.Params.NodeMethod = 0
model.optimize()
# Check the optimization status
if model.status == gugu.GRB.OPTIMAL:
    # Access the values of the decision variables
    x_value = x.x
    y1_value = y1.x
    z1_value = z1.x
    y2_value = y2.x
    z2_value = z2.x

    print("Optimal Solution:")
    print('Obj: %g' % model.objVal)
    print(f"x = \n{x_value}")
    print("\nScenarios 1:")
    print(f"y1 = \n{y1_value}")
    print(f"\nz1 = \n{z1_value}")
    print("\n\nScenarios 2:")
    print(f"y2 = \n{y2_value}")
    print(f"\nz2 = \n{z2_value}")
    model.printAttr('x')
else:
    print("Optimization was not successful.")

Optimal Solution:
Obj: -1099
x = 
[[140.]
 [170.]
 [108.]
 [155.]
 [151.]]

Scenarios 1:
y1 = 
[[3.]
 [0.]
 [0.]
 [0.]
 [0.]]

z1 = 
[[3.]
 [3.]
 [2.]
 [5.]
 [2.]
 [7.]
 [7.]
 [2.]]


Scenarios 2:
y2 = 
[[0.]
 [0.]
 [6.]
 [0.]
 [0.]]

z2 = 
[[5.]
 [3.]
 [4.]
 [5.]
 [1.]
 [7.]
 [1.]
 [5.]]
